In [1]:
from mlx_lm import load, generate
import json
from tqdm import tqdm
import csv

In [ ]:
OUT_CSV = './data/eval_out.csv'

In [8]:
# Load test data
with open("data/processed/test.jsonl", "r") as f:
    test_data = [json.loads(line) for line in f]

In [4]:
print('Loading base model...')
model_base, tokenizer_base = load('./models/base/Llama-3.2-1B-Instruct-4bit')

print('Loading fine-tuned model...')
model_ft, tkenizer_ft= load('./models/fused/fused_Llama-3.2-1B-Instruct-4bit_2bs_4ls')

Loading base model...
Loading fine-tuned model...


In [ ]:
# prepare OUT CSV
with open(OUT_CSV, 'w', newline='') as csvfile:
	csv_writer = csv.writer(csvfile)
	# Write the header
	csv_writer.writerow(['Streamer', '#', 'Base - Question Prompt', 'Finetuned - Question Prompt'])
